In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import torch
import torch.nn as nn
import torchvision
from PIL import Image
import json
import os
from tqdm.notebook import tqdm

In [2]:
!nvidia-smi

In [3]:
!pip install gdown

In [9]:
!gdown "https://drive.google.com/uc?id=1-0TjTRNHiuz5e2FlDqF4Uw6T4JVsXxMq"

In [11]:
!unzip -q "./Medical+mask.zip"

In [12]:
# picture: torch.Tensor or np.array
# boxes: indexed object
# labels: indexed object
def draw_pic_with_rect(picture, boxes, labels, n=5):
    boxes = boxes[:n]
    labels = labels[:n]
    if isinstance(picture, torch.Tensor):
        picture = (picture.detach().squeeze(0) * 256).permute(1, 2, 0).numpy()
    picture = picture.astype(dtype=np.int)

    fig, ax = plt.subplots(figsize = (15, 15))
    ax.imshow(picture)

    for box, lab in zip(boxes, labels):
        rect = matplotlib.patches.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1],
                                       linewidth=1, edgecolor='r', facecolor='none')
        ax.text(box[0], box[1], labels, fontsize = 12)
        ax.add_patch(rect)

    fig.show()

In [13]:
!nvidia-smi

In [14]:
img = Image.open("./Medical mask/Medical Mask/images/0010.jpg")
img = np.asarray(img)
plt.imshow(img)

In [ ]:
draw_pic_with_rect(img, [[541, 394, 691, 504], [759, 353, 879, 490]], "mask")

In [ ]:
with open("/content/Medical mask/Medical Mask/annotations/0001.jpg.json") as f:
    fff = json.loads(f.read())
fff

In [15]:
labels_transform = {
    "face_with_mask": 2,
    "mask_colorful": 1,
    "mask_surgical": 1,
    "face_no_mask": 2,
    "face_with_mask_incorrect": 2,
    "face_other_covering": 2,
}

In [16]:
targets = []

cnt = 0
path_annotations = "./Medical mask/Medical Mask/annotations"
files = os.listdir(path_annotations)
files.sort()

for file in tqdm(files):
    cnt += 1
    real_path = os.path.join(path_annotations, file)
    dick = None
    with open(real_path, "r") as f:
        dick = json.loads(f.read())
    element = {
        "boxes": [],
        "labels": [],
        "filename": dick["FileName"],
    }

    for annotation in dick["Annotations"]:
        if annotation["classname"] in list(labels_transform.keys()):
            element["boxes"].append(annotation["BoundingBox"])
            element["labels"].append(labels_transform[annotation["classname"]])
            
    element["boxes"] = torch.tensor(element["boxes"])
    element["labels"] = torch.tensor(element["labels"])
    
    if len(element["labels"]) > 0:
        targets.append(element)

In [17]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [18]:
from torch.utils.data import DataLoader

targets_loader = DataLoader(targets, batch_size=4, shuffle=True, drop_last=True, collate_fn=lambda x: x)

In [19]:
torch.manual_seed(42)
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

model.to(device)
params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(params, lr=1e-4)

In [20]:
# The training loop trains the model for the total number of epochs.
# (1 epoch = one complete pass over the entire dataset)
num_epochs = 5
path_images = "./Medical mask/Medical Mask/images"
transform = torchvision.transforms.Compose([
    torchvision.transforms.PILToTensor(),                                           
])
model.train()
epoch_loss = []
for epoch in range(num_epochs):
    pbar = tqdm(targets_loader)
    _train_loss = []
    success_iter = 0
    for num, batch in enumerate(pbar):
        images = []

        for el in batch:
            picture_pil = Image.open(os.path.join(path_images, el["filename"]))
            images.append(transform(picture_pil).to(device, dtype=torch.float32))
        
        t = [{"boxes": el["boxes"].to(device, dtype=torch.int64), 
              "labels": el["labels"].to(device, dtype=torch.int64),
              } for el in batch]
        try:
            loss_dict = model(images, t)
        except ValueError:
            continue
        success_iter += 1

        losses = sum(loss for loss in loss_dict.values())

        _train_loss.append(losses.item())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step() 
        del images
        del t
        torch.cuda.empty_cache()
        if num % 50 == 0:
            epoch_loss.append(np.mean(_train_loss))
            pbar.set_description(f"loss_value: {np.mean(_train_loss)}")
            _train_loss = []
    
    print(success_iter)
    torch.save(model.state_dict(), f"xyi{epoch + 2}.pt")
    plt.plot(np.arange(len(epoch_loss)), np.array(epoch_loss))

In [35]:
# picture: torch.Tensor or np.array
# boxes: indexed object
# labels: indexed object
def draw_pic_with_rect(picture, boxes, labels, n=5):
    boxes = boxes[:n]
    labels = labels[:n]
    if isinstance(picture, torch.Tensor):
        picture = picture.detach().cpu().permute(1, 2, 0).numpy()
    picture = picture.astype(dtype=np.int)

    fig, ax = plt.subplots(figsize = (15, 15))
    ax.imshow(picture)

    for box, label in zip(boxes, labels):
        if label == 2:
            continue
        rect = matplotlib.patches.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1],
                                       linewidth=1, edgecolor='r', facecolor='none')
        ax.text(box[0], box[1], label, fontsize = 12)
        ax.add_patch(rect)

    fig.show()

In [22]:
model.eval();

In [45]:
picture_pil = Image.open("./Medical mask/Medical Mask/images/0009.jpg")
image = transform(picture_pil).to(device, dtype=torch.float32)
preds = model(image.unsqueeze(0))
preds

In [48]:
draw_pic_with_rect(image.cpu().detach().to(dtype=torch.int), preds[0]["boxes"].cpu().detach(), preds[0]["labels"].cpu().detach().tolist(), 5)

In [65]:
picture_pil = Image.open("../input/face-mask-detection/images/maksssksksss1.png")
image = transform(picture_pil).to(device, dtype=torch.float32)[:3, :, :]
preds = model(image.unsqueeze(0))
draw_pic_with_rect(image.cpu().detach().to(dtype=torch.int), preds[0]["boxes"].cpu().detach(), preds[0]["labels"].cpu().detach().tolist(), 15)

In [66]:
!gdown --folder --id "1kLAh4-si-_8TU7b_P-rTtzlqTyB2CjqD"

In [68]:
!ls Tulahack

In [70]:
picture_pil = Image.open("Tulahack/img1.png")
image = transform(picture_pil).to(device, dtype=torch.float32)[:3, :, :]
preds = model(image.unsqueeze(0))
draw_pic_with_rect(image.cpu().detach().to(dtype=torch.int), preds[0]["boxes"].cpu().detach(), preds[0]["labels"].cpu().detach().tolist(), 20)

In [75]:
picture_pil = Image.open("Tulahack/seva1.jpg")
image = transform(picture_pil).to(device, dtype=torch.float32)[:3, :, :]
preds = model(image.unsqueeze(0))
draw_pic_with_rect(image.cpu().detach().to(dtype=torch.int), preds[0]["boxes"].cpu().detach(), preds[0]["labels"].cpu().detach().tolist(),5)

In [76]:
picture_pil = Image.open("Tulahack/sev2.jpg")
image = transform(picture_pil).to(device, dtype=torch.float32)[:3, :, :]
preds = model(image.unsqueeze(0))
draw_pic_with_rect(image.cpu().detach().to(dtype=torch.int), preds[0]["boxes"].cpu().detach(), preds[0]["labels"].cpu().detach().tolist(), 20)pg")
image = transform(picture_pil).to(device, dtype=torch.float32)[:3, :, :]
preds = model(image.unsqueeze(0))